In [20]:
import os
import torch
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                        AutoTokenizer,
                        BitsAndBytesConfig,
                        HfArgumentParser,
                        TrainingArguments,
                        Pipeline,
                        logging
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

in case of Llama 2 , the following prompt template is used for the chat models

system Prompt(optional) to guide the model

user prompt (required) to give the instruction 

Model Answer (required) 

we will reformat our instruction dataset to follow Llama 2 template.

* Original dataset https://huggingface.co/datasets/timdettmers/openassistant-guanaco

* Reformat Dataset following the Llama 2 template with 1k sample:https://huggingface.co/datasets/mlabonne/guanaco-llama2-1k 

* Complete Reformat Dataset following the Llama2 template: https://huggingface.co/datasets/mlabonne/guanaco-llama2


How to fine tune llama2

* free Google Cloud offers a 15GB Graphics Card(Limited Resources--> Barely enough to store Llama 2 -7b's weights)

* we also need to consider the overhead due to optimizer states, gradients, and forward activations

* Full fine-tunning is not possible here: we need parameter-efficient fine-tuning(PEFT) techniques like LoRA or QLoRA

* To drastically reduce the VRAM usage, we must fine-tune the model in 4-bit precision, which is why we'll use QLoRA here.

step 3

1. Load a llama-2-7b-chat-hf model (chat models)

2. Train it on the mlabonne/guanaco-llama2k(1,000 samples), which will produce our fine tunined model Llama-2-7b-chat-finetune

QLoRA will use a rank of 64 with a scalling of 16. we'll load the Llama 2 model directly in 4-bit precision using the NF4 tykpe and train it for one epoch

In [24]:
# the model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# the instruction  dataset to use
dataset_name = "mlabonne/guanaco-llama2-1k"
# Fine-tunned model name
new_model = "Llama-2-7b-chat-finetune"

##########################################################
# QLoRA parameters
##########################################################

# LoRA attention dimensions
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

##########################################################
# bitsandbytes parameters
##########################################################

# Activate 4-bit precision base model loading 
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type ( fp4 or nf4 )
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False


########################################################################
# TrainingArguments parameters 
########################################################################

# Output directory where the model predictions and checkpoints will be stored.
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with and A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation

per_device_eval_batch_size = 4


# Number of updates steps to accumulate the gradients for

gradient_accumulation_steps = 1

# Enable gradient checkpoint
gradient_checkpointing = True

# Initial learning rate (AdamW optimizer)

learning_rate = 2e-4

# weights decay to apply to all layers except bias/LayerNorm weights

weight_decay = 0.001

# Optimizer to use 
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
warmup_ratio = 0.03 

# Group sequence into batches with same length
# saves memory and speeds up training considerably
group_by_length = True

# save checkpoints every X updates steps
save_steps = 0

# Log every X update steps
logging_steps = 25

###########################################################
# SFT parameters
###########################################################

# maximum sequence length to use 
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}



Step 4: Load everything and start the fine-tuning process

1. First of all, we want to load the dataset we defined. Here , out dataset is already preprocessed but , usually, this is whre you would
reformat the prompt , filter out bad text , combine multiole datasets , etc.

2. then , we're configuring bitsandbytes for 4-bit quantization.

3. Next, we're loading the Llama 2 model in 4-bit precision on a GPU with the corresponding tokenizer

4. Finally, we're loading configurations for QLaRA , regular training parameters, and passing everything to the SFTTrainer. The Training can finally start.

In [25]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch , bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# check GPU compatibility with bfloat6
if compute_dtype == torch.float64 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("="*80)
        print("your GPU supports bfloat16: accelerate training with bf16=true")
        print("="*80)

# Load base model from huggingFace

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issie with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
)

# set training parameters
training_arguments = TrainingArguments(
output_dir= output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    #per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    gradient_checkpointing=gradient_checkpointing,
    learning_rate=learning_rate,
    num_train_epochs= num_train_epochs,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    weight_decay=weight_decay,
    optim=optim,
    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    save_steps=save_steps,
    logging_steps=logging_steps,
    fp16=fp16,
    bf16=bf16,
    report_to="tensorboard",

)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config= peft_config,
    dataset_text_field='text',
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

Generating train split: 100%|██████████| 1000/1000 [00:00<00:00, 11975.00 examples/s]
c:\Users\hp26\Documents\MCQGenerator\env\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp26\.cache\huggingface\hub\models--NousResearch--Llama-2-7b-chat-hf. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


RuntimeError: No GPU found. A GPU is needed for quantization.

In [19]:
# save trained model
trainer.model.save_pretrained(new_model)

NameError: name 'trainer' is not defined

step 5: check the plots on tensorbard, as follows


In [ ]:
%load_ext tensorbard
%tensorbard --logdir results/run


step 6: Use the text generation pipeline to ask questions like "What is LLM?", Note that i'm formatting the input to match Llama2 prompt text

In [21]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "what is a large language model?"
pipe = Pipeline(task = "text-generation",model = model, tokenizer = tokenizer, max_seq_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]") # INST is instruction 
print(result[0]['generated_text'])


NameError: name 'model' is not defined

In [ ]:
# empty VRAM


you can train a Llama 2 model on the entire dataset using mlabanne/guanaco-llama2



step 7: Store New Llama2 Model (Llama-2-7b-chat-finetune)

How can we store our new Llama2-7b-chat-finetune model now? we need to merge the weights from LoRA with the base model. Unfortunately, as far as i know, there is no straightforward way to do it: we need to reload the base model in FP16 precision and use the peft library to merge everything.

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage = True,
    return_dict = True,
    torch_dtype = torch.float16,
    device_map = device_map,
)
model1 = PeftModel.from_pretrained(base_model, new_model)
model2 = model.merge_and_unload()

# Reload tokenizer to save it

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issie with fp16 training

# Save model


step 8 : Push Model to Hugging Face Hub

Our weights are merged and we reloaded the tokenizer. we can now push everything to the Hugging Face Hub to save our model.

import locale
locale.getpreferredencoding = lambda: "UTF-8"


In [ ]:
!huggingface-cli logging

model.push_to_hub("entbappy/Llama-2-7b-chat-finetune", check_pr = True)
tokenizer.push_to_hub("entbappy/Llama-2-7b-finetune", check_pr = True)